<a href="https://colab.research.google.com/github/agmmnn/multisubd/blob/master/multi_subd.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### [![](https://raw.githubusercontent.com/agmmnn/awesome-blender/master/imgs/github.svg) github.com/agmmnn/multisub](https://github.com/agmmnn/multisub)

#### Data Content
- `.json` result of httpx query. It contains information such as timestamp, hash, webserver, host informations and so on.
- `.txt` files contains only urls.

In [ ]:
!yes | sudo add-apt-repository universe &> /dev/null
!yes | sudo apt update &> /dev/null
!yes | sudo apt install p7zip-full p7zip-rar &> /dev/null
!sudo apt-get install libpcap0.8 &> /dev/null

In [9]:
!wget $(curl -s https://api.github.com/repos/owasp-amass/amass/releases/latest | grep browser_download_url | grep 'Linux_amd64.zip' | head -n 1 | cut -d '"' -f 4) &> /dev/null && yes | unzip "amass_Linux_amd64.zip" -d "./tools" &> /dev/null && rm "amass_Linux_amd64.zip"


In [ ]:
!wget $(curl -s https://api.github.com/repos/owasp-amass/amass/releases/latest | grep browser_download_url | grep 'Linux_amd64.zip' | head -n 1 | cut -d '"' -f 4)
!yes | unzip amass*.zip -d "./tools" && rm amass*.zip

!wget $(curl -s https://api.github.com/repos/projectdiscovery/subfinder/releases/latest | grep browser_download_url | grep 'linux_amd64.zip' | head -n 1 | cut -d '"' -f 4)
!yes | unzip subfinder*.zip -d "./tools" && rm subfinder*.zip

!wget $(curl -s https://api.github.com/repos/projectdiscovery/httpx/releases/latest | grep browser_download_url | grep 'linux_amd64.zip' | head -n 1 | cut -d '"' -f 4)
!yes | unzip httpx*.zip -d "./tools" && rm httpx*.zip

In [ ]:
!chmod u=rwx,g=r,o=r ./tools/xray_linux_amd64

In [ ]:
domain_list = ["health-tourism.com","mymeditravel.com","flymedi.com","whatclinic.com","bookimed.com","bookinghealth.com","booking.dentist"]

#### `1. Domain Enumeration`

_Run it after uploading the config files or just delete the config commands(-config config.ini, -pc provider-config.yaml)._

In [ ]:
import subprocess
import time
from concurrent.futures import ProcessPoolExecutor

start_total_time = time.perf_counter()

# Create a ProcessPoolExecutor with 4 workers
with ProcessPoolExecutor(max_workers=4) as executor:
    future_list = []
    for domain in domain_list:
        start_time = time.perf_counter()
        print(f"📄{domain}.amass.txt, {domain}.subfinder.txt, {domain}.xray.txt")
        #!./tools/amass_Linux_amd64/amass enum -passive -d "$domain" -o "$domain".amass.txt -silent -config "config.ini" &> /dev/null
        amass_cmd = [
            "./tools/amass_Linux_amd64/amass",
            "enum",
            "-passive",
            "-d",
            domain,
            "-o",
            f"{domain}.amass.txt",
            "-silent",
            "-config",
            "./config.ini",
        ]
        #!./tools/subfinder -d $domain -o "$domain".subfinder.txt -all -t 20 -nW -silent -max-time 120 -timeout 6  -pc "provider-config.yaml" &> /dev/null
        subfinder_cmd = [
            "./tools/subfinder",
            "-d",
            domain,
            "-o",
            f"{domain}.subfinder.txt",
            "-all",
            "-t",
            "20",
            # "-nW",
            "-silent",
            "-max-time",
            "120",
            "-timeout",
            "6",
            "-pc",
            "./provider-config.yaml",
        ]

        xray_cmd = [
            "./tools/xray_linux_amd64",
            "sd",
            "x",
            "-t",
            domain,
            "--text-output",
            f"{domain}.xray.txt",
            "--no-brute"
        ]

        #Submit the subprocesses to the executor and store the future objects in a list
        future_list.append(executor.submit(subprocess.run, " ".join(amass_cmd),shell=True))
        future_list.append(executor.submit(subprocess.run, " ".join(subfinder_cmd),shell=True))
        future_list.append(executor.submit(subprocess.run, " ".join(xray_cmd),shell=True))

    # Wait for all the submitted subprocesses to complete
    for future in future_list:
        future.result()

    end_time = time.perf_counter()
    execution_time = end_time - start_time
    print(f"Execution time: {int(execution_time)}s")

end_total_time = time.perf_counter()
execution_total_time = end_total_time - start_total_time
print(f"\nTotal execution time: {int(execution_total_time)}s")

#### `2. Merge lists, clean non-root domains`

In [ ]:
for domain in domain_list:
    # merge files/lists
    inputlist = []
    inputfiles = [domain + ".amass.txt", domain + ".subfinder.txt"]
    for i in inputfiles:
        with open(i, "r", encoding="utf-8") as f:
            inputlist.extend(f.readlines())
    with open(domain + ".xray.txt", "r", encoding="utf-8") as f:
        xlist=f.readlines()
        for x in xlist:
            inputlist.append(x.split(",")[0]+"\n")


    inputlist=list(set(inputlist))
    with open(domain + "_all_final.txt", "w", encoding="utf-8") as f:
        for i in inputlist:
            f.write(i)
    print(f"💾{domain}_all_final.txt: {len(inputlist)}")

    # cleaning to root domains
    itemlist = []
    for i in inputlist:
        if "xn--" in i:
            continue
        item = i.split(f".{domain}")[0].split(".")[-1] + f".{domain}"
        if item not in itemlist:
            itemlist.append(item)

    # save domains: domain_final.txt
    itemlist.sort()
    with open(domain + "_final.txt", "w", encoding="utf-8") as f:
        if not domain == "tsk.tr":
            for i in itemlist:
                f.write(f"{i}\n")
        else:
            for i in itemlist:
                if i != "www.tsk.tr":
                    f.write(f"www.{i}\n")
                else:
                    f.write(f"{i}\n")
    print(f"💾{domain}_final.txt: {len(itemlist)}")

#### `3. Validating the domain list with httpx`

In [ ]:
from concurrent.futures import ProcessPoolExecutor

child_processes = []

for domain in domain_list:
    print(f"📄{domain}_all_final.jsonl")
    p = subprocess.Popen(
        [
            "./tools/httpx",
            "-l",
            f"{domain}_all_final.txt",
            "-t",
            "100",
            "-timeout",
            "7",
            "-json",
            "-o",
            f"{domain}_final.jsonl",
            "-silent",
            "-td"
        ],
        stdout=subprocess.DEVNULL
    )
    child_processes.append(p)

with ProcessPoolExecutor(max_workers=3) as executor:
    for cp in child_processes:
        executor.submit(cp.wait)

#### `4. Generating json file and url list from httpx output`

In [ ]:
for domain in domain_list:
    import json

    with open(domain + "_final.jsonl", encoding="utf-8") as f:
        data = [json.loads(line) for line in f]

    print(f"🌐{domain}:", len(data))
    out = json.dumps(data)  # .encode("utf-8").decode('unicode-escape')

    with open(domain + ".json", "w", encoding="utf-8") as f:
        f.write(out)
    print(f"\t💾 {domain}.json")

    # json to urls.txt
    with open(domain + ".json", "r", encoding="utf-8") as f:
        jdata = json.load(f)

    urls = []
    for i in jdata:
        item = i["url"].split("//")[1].split(":")[0]
        if item not in urls:
            urls.append(item)
    urls.sort()
    with open(domain + ".txt", "w", encoding="utf-8") as f:
        for i in urls:
            f.write("%s\n" % i)
    print(f"\t💾 {domain}.txt")

# final zip
import datetime
final_files = (
    ".txt ".join(domain_list) + ".txt" + " " + ".json ".join(domain_list) + ".json"
)
all_files="_final.txt ".join(domain_list) + "_final.txt" + " " + "_all_final.txt ".join(domain_list) + "_all_final.txt" + " " + final_files
date = datetime.datetime.now().strftime("%Y_%m_%d")
zipfile = f"{date}_final.7z"
zipallfile = f"{date}_final_all.7z"
print(f"\n🗃️ {zipfile}")
!7z a -t7z -m0=lzma2 -mx=9 -mfb=64 -md=32m -ms=on $zipfile $final_files &> /dev/null
print(f"🗃️ {zipallfile}")
!7z a -t7z -m0=lzma2 -mx=9 -mfb=64 -md=32m -ms=on $zipallfile $all_files &> /dev/null